Задание 18.10


In [1]:
import random
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.misc import derivative

#Номер 1


In [2]:

# Определяем функцию f
def f(x):
    return np.exp(-np.dot(x, x)) * x

# Метод конечных разностей
def finite_difference(f, x, alpha):
    return (f(x + alpha) - f(x)) / alpha

# Метод комплексного приращения
def complex_step(f, x, alpha):
    return np.imag(f(x + 1j * alpha)) / alpha

# Автоматическое дифференцирование (используем библиотеку autograd)
from autograd import grad

# Задаем предел относительно погрешности входных данных
tolerance = 0.001

# Основная функция для вычисления производной
def compute_derivatives(x, p_values):
    finite_diff_results = []
    complex_step_results = []
    auto_diff_results = []

    for p in p_values:
        alpha = 10 ** -p

        # Оценка производной методом конечных разностей
        finite_diff_result = finite_difference(f, x, alpha)
        finite_diff_results.append(finite_diff_result)

        # Оценка производной методом комплексного приращения
        complex_step_result = complex_step(f, x, alpha)
        complex_step_results.append(complex_step_result)

        # Оценка производной методом авто-дифференцирования
        auto_diff_function = grad(f)
        auto_diff_result = auto_diff_function(x)
        auto_diff_results.append(auto_diff_result)

    return finite_diff_results, complex_step_results, auto_diff_results

# Начальные условия
x = np.zeros(5)  # Пример с 5 аргументами
p_values = range(-300, 1)

# Рассчитываем производные для разных p
finite_diff_results, complex_step_results, auto_diff_results = compute_derivatives(x, p_values)

# Построение графиков
plt.figure(figsize=(10, 6))
plt.plot(p_values, finite_diff_results, label='Конечные разности', color='r')
plt.plot(p_values, complex_step_results, label='Комплексные приращения', color='g')
plt.plot(p_values, auto_diff_results, label='Автоматическое дифференцирование', color='b')
plt.yscale('log')
plt.xlabel('p (где α = 10^{-p})')
plt.ylabel('Оценка производной')
plt.title('Оценки производной функции f')
plt.legend()
plt.grid()
plt.show()

# Оценка погрешностей
# Применим линеаризацию
def linearization_error(f, x, tolerance):
    grad_f = grad(f)  # Градиент функции f
    error = np.linalg.norm(grad_f(x) * tolerance)
    return error

# Погрешности
error_estimation = linearization_error(f, x, tolerance)
print(f"Наследственная погрешность значения функции f: {error_estimation}")

<ipython-input-2-6424e8619d99>:3: RuntimeWarning: invalid value encountered in multiply
  return np.exp(-np.dot(x, x)) * x


TypeError: loop of ufunc does not support argument 0 of type ArrayBox which has no callable exp method

#Номер 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Интервальная арифметика
class Interval:
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper

    def __add__(self, other):
        return Interval(self.lower + other.lower, self.upper + other.upper)

    def __sub__(self, other):
        return Interval(self.lower - other.upper, self.upper - other.lower)

    def __mul__(self, other):
        return Interval(
            min(self.lower * other.lower, self.lower * other.upper,
                self.upper * other.lower, self.upper * other.upper),
            max(self.lower * other.lower, self.lower * other.upper,
                self.upper * other.lower, self.upper * other.upper)
        )

    # Метод для вычисления функции f
    def f(self):
        return Interval(np.exp(-self.upper**2), np.exp(-self.lower**2)) # простая оценка для f(x)

# Аффинная арифметика
class Affine:
    def __init__(self, coeff, constant):
        self.coeff = coeff
        self.constant = constant

    def __add__(self, other):
        return Affine(self.coeff + other.coeff, self.constant + other.constant)

    def __sub__(self, other):
        return Affine(self.coeff - other.coeff, self.constant - other.constant)

    def __mul__(self, other):
        return Affine(self.coeff * other.coeff, self.coeff * other.constant + self.constant * other.coeff)

    def f(self):
        return Affine(self.coeff * np.exp(-self.constant**2), 0) # аффинная оценка для f(x)

# Подсчет наследственной погрешности
def compute_hereditary_error(interval, affine):
    return (interval.upper - interval.lower), (affine.coeff)

# Основной расчёт
def main():
    results = {}
    alpha_values = []

    for p in range(-300, 1):  # от -300 до 0
        alpha = 10 ** -p
        alpha_values.append(alpha)

        interval_result = Interval(-alpha, alpha).f()
        affine_result = Affine(-alpha, alpha).f()

        # Рассчет наследственной погрешности
        interval_error, affine_error = compute_hereditary_error(interval_result, affine_result)

        results[p] = {
            'alpha': alpha,
            'interval_result': interval_result,
            'affine_result': affine_result,
            'interval_error': interval_error,
            'affine_error': affine_error
        }

    # Сравнение результатов
    for p in results.keys():
        print(f"p: {p}, α: {results[p]['alpha']}, "
              f"Interval Result: [{results[p]['interval_result'].lower}, {results[p]['interval_result'].upper}], "
              f"Affine Result: [{results[p]['affine_result'].coeff + results[p]['affine_result'].constant}], "
              f"Interval Error: {results[p]['interval_error']}, "
              f"Affine Error: {results[p]['affine_error']}")

main()

TypeError: loop of ufunc does not support argument 0 of type int which has no callable exp method